In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    # driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [3]:
import urllib
from configparser import ConfigParser

config = ConfigParser()
config.read('config_private.ini')

client_id = "1fd1295e25e841f39b73d0fab94962bb"
client_secret = "b1f1cd3dcf9d4992bd022e0029250b96"
redirect_uri = "http://localhost:8888/callback"

scope = "playlist-modify-public"

params = {
    "response_type": "code",
    "client_id": client_id,
    "scope": scope,
    "redirect_uri": redirect_uri,
    "state": "abcdefghijklmnop"  # some random 16 digit string? - needed?
}
auth_params = urllib.parse.urlencode(query=params)
auth_url = "https://accounts.spotify.com/authorize"

In [29]:


driver = driversetup()
driver.get(f"{auth_url}?{auth_params}")
driver.find_element(By.ID, "login-username").send_keys(config["SPOTIFY"]["username"])
driver.find_element(By.ID, "login-password").send_keys(config["SPOTIFY"]["password"])
driver.find_element(By.ID, "login-button").click()


In [32]:
driver = webdriver.Chrome()
driver.get(f"{auth_url}?{auth_params}")

driver.find_element(By.ID, "login-username").send_keys(config["SPOTIFY"]["username"])
driver.find_element(By.ID, "login-password").send_keys(config["SPOTIFY"]["password"])

driver.find_element(By.ID, "login-button").click()

# TODO: Check that we haven't already authorized - or we won't hit this redirect to confirm
driver.find_element(By.XPATH, "//button[@data-testid='auth-accept']").click()

parsed = urllib.parse.urlparse(driver.current_url)
code = urllib.parse.parse_qs(parsed.query)["code"]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-testid='auth-accept']"}
  (Session info: chrome=107.0.5304.110)


In [22]:
token_url = "https://accounts.spotify.com/api/token"
token_data = {
  "code": code,
  "redirect_uri": redirect_uri,
  "grant_type": "authorization_code",
}
headers = {
  # "Authorization": "Basic " + (new Buffer(client_id + ':' + client_secret).toString('base64')),
}

r = requests.post(
    url=token_url, 
    data=token_data,
    auth=requests.auth.HTTPBasicAuth('client_id', 'client_secret'),
)

In [31]:
driver.close()